# Camera Calibration with OpenCV and Gradio

This notebook implements camera calibration using OpenCV with an interactive Gradio interface. The system computes camera intrinsic parameters (focal length, principal point, distortion coefficients) from chessboard calibration images.

## Setup and Dependencies

Install all required packages:

In [3]:
# Install required packages
%pip install opencv-python numpy matplotlib plotly gradio pillow
%pip install pytransform3d  # Optional for advanced 3D visualization

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Home

## Import Libraries and Helper Classes

In [5]:
import gradio as gr
import numpy as np
import cv2
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import json
import os
import shutil
from PIL import Image
from typing import List, Tuple, Optional
import zipfile
import tempfile
import glob

# Import our helper classes
try:
    # Try to import from the same directory
    from calibration_utils import Calib, IO, Img, Overlay, Render
    print("✅ Imported calibration utilities from external file")
except (ImportError, FileNotFoundError):
    # If external file not found, classes will be defined in next cell
    print("⚠️ calibration_utils.py not found, using inline definitions")
    pass

⚠️ calibration_utils.py not found, using inline definitions


## Helper Classes (Inline Definition for Colab)

If you're running this in Google Colab, the helper classes will be defined here:

In [11]:
# Define helper classes inline if not already imported
if 'Calib' not in globals():
    print("Loading helper classes inline...")
    
    # For Colab compatibility, we'll embed the classes here
    import cv2
    import numpy as np
    import json
    import os
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    import plotly.graph_objects as go
    from typing import List, Tuple, Optional
    import glob


    class IO:
        """File and device utilities"""
        
        @staticmethod
        def create_directory(path: str) -> None:
            """Create directory if it doesn't exist"""
            os.makedirs(path, exist_ok=True)
        
        @staticmethod
        def list_images(directory: str, extension: str = "*.jpg") -> List[str]:
            """List all images in directory with given extension"""
            jpg_files = glob.glob(os.path.join(directory, extension))
            jpeg_files = glob.glob(os.path.join(directory, "*.jpeg"))
            return jpg_files + jpeg_files
        
        @staticmethod
        def save_calibration_results(filename: str, camera_matrix: np.ndarray, 
                                   dist_coeffs: np.ndarray, rvecs: List[np.ndarray], 
                                   tvecs: List[np.ndarray], image_size: Tuple[int, int]) -> None:
            """Save calibration results to JSON file"""
            results = {
                'camera_matrix': camera_matrix.tolist(),
                'distortion_coefficients': dist_coeffs.tolist(),
                'rotation_vectors': [rvec.tolist() for rvec in rvecs],
                'translation_vectors': [tvec.tolist() for tvec in tvecs],
                'image_width': int(image_size[0]),
                'image_height': int(image_size[1])
            }
            
            with open(filename, 'w') as f:
                json.dump(results, f, indent=4)
        
        @staticmethod
        def load_calibration_results(filename: str) -> dict:
            """Load calibration results from JSON file"""
            with open(filename, 'r') as f:
                return json.load(f)


    class Board:
        """Chessboard object model"""
        
        def __init__(self, pattern_size: Tuple[int, int], square_size: float):
            self.pattern_size = pattern_size
            self.square_size = square_size
            self.objp = self._create_object_points()
        
        def _create_object_points(self) -> np.ndarray:
            """Create 3D object points for chessboard pattern"""
            objp = np.zeros((self.pattern_size[0] * self.pattern_size[1], 3), np.float32)
            objp[:, :2] = np.mgrid[0:self.pattern_size[0], 0:self.pattern_size[1]].T.reshape(-1, 2)
            objp *= self.square_size
            return objp
        
        @staticmethod
        def find_corners(image: np.ndarray, pattern_size: Tuple[int, int]) -> Tuple[bool, Optional[np.ndarray]]:
            """Find chessboard corners in image"""
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
            
            if ret:
                criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
                corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            
            return ret, corners


    class Img:
        """Image utilities"""
        
        @staticmethod
        def load_image(filepath: str) -> np.ndarray:
            """Load image from file"""
            return cv2.imread(filepath)
        
        @staticmethod
        def resize_image(image: np.ndarray, width: int = 800) -> np.ndarray:
            """Resize image maintaining aspect ratio"""
            height, orig_width = image.shape[:2]
            aspect_ratio = height / orig_width
            new_height = int(width * aspect_ratio)
            return cv2.resize(image, (width, new_height))
        
        @staticmethod
        def undistort_image(image: np.ndarray, camera_matrix: np.ndarray, 
                           dist_coeffs: np.ndarray) -> np.ndarray:
            """Undistort image using calibration parameters"""
            return cv2.undistort(image, camera_matrix, dist_coeffs, None, camera_matrix)


    class Overlay:
        """Draw axes overlays and visualization"""
        
        @staticmethod
        def draw_axes(image: np.ndarray, camera_matrix: np.ndarray, dist_coeffs: np.ndarray,
                      rvec: np.ndarray, tvec: np.ndarray, length: float = 0.1) -> np.ndarray:
            """Draw 3D coordinate axes on image"""
            axes_points = np.float32([[0,0,0], [length,0,0], [0,length,0], [0,0,-length]]).reshape(-1,3)
            
            imgpts, _ = cv2.projectPoints(axes_points, rvec, tvec, camera_matrix, dist_coeffs)
            imgpts = np.int32(imgpts).reshape(-1, 2)
            
            img_with_axes = image.copy()
            corner = tuple(imgpts[0].ravel().astype(int))
            
            img_with_axes = cv2.line(img_with_axes, corner, tuple(imgpts[1].ravel().astype(int)), (0,0,255), 5)
            img_with_axes = cv2.line(img_with_axes, corner, tuple(imgpts[2].ravel().astype(int)), (0,255,0), 5)
            img_with_axes = cv2.line(img_with_axes, corner, tuple(imgpts[3].ravel().astype(int)), (255,0,0), 5)
            
            return img_with_axes
        
        @staticmethod
        def draw_chessboard_corners(image: np.ndarray, corners: np.ndarray, 
                                   pattern_size: Tuple[int, int], found: bool) -> np.ndarray:
            """Draw detected chessboard corners"""
            img_with_corners = image.copy()
            cv2.drawChessboardCorners(img_with_corners, pattern_size, corners, found)
            return img_with_corners


    class Cam:
        """Camera math and calibration utilities"""
        
        @staticmethod
        def calibrate_camera(object_points: List[np.ndarray], image_points: List[np.ndarray],
                            image_size: Tuple[int, int]) -> Tuple[float, np.ndarray, np.ndarray, 
                                                                 List[np.ndarray], List[np.ndarray]]:
            ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
                object_points, image_points, image_size, None, None)
            
            return ret, camera_matrix, dist_coeffs, rvecs, tvecs
        
        @staticmethod
        def compute_reprojection_error(object_points: List[np.ndarray], image_points: List[np.ndarray],
                                      camera_matrix: np.ndarray, dist_coeffs: np.ndarray,
                                      rvecs: List[np.ndarray], tvecs: List[np.ndarray]) -> float:
            """Compute mean reprojection error"""
            total_error = 0
            total_points = 0
            
            for i in range(len(object_points)):
                projected_points, _ = cv2.projectPoints(object_points[i], rvecs[i], tvecs[i], 
                                                      camera_matrix, dist_coeffs)
                error = cv2.norm(image_points[i], projected_points, cv2.NORM_L2) / len(projected_points)
                total_error += error
                total_points += 1
            
            return total_error / total_points if total_points > 0 else 0


    class Render:
        """Visualization and rendering utilities"""
        
        @staticmethod
        def plot_camera_poses_plotly(rvecs: List[np.ndarray], tvecs: List[np.ndarray], 
                                    board_size: Tuple[int, int], square_size: float):
            """Plot camera poses using plotly for interactive visualization"""
            fig = go.Figure()
            
            # Plot chessboard points
            board_points = []
            for i in range(board_size[1] + 1):
                for j in range(board_size[0] + 1):
                    board_points.append([j * square_size, i * square_size, 0])
            
            board_points = np.array(board_points)
            fig.add_trace(go.Scatter3d(
                x=board_points[:, 0], y=board_points[:, 1], z=board_points[:, 2],
                mode='markers',
                marker=dict(size=3, color='red'),
                name='Chessboard',
                opacity=0.6
            ))
            
            # Plot camera positions
            camera_positions = []
            for i, (rvec, tvec) in enumerate(zip(rvecs, tvecs)):
                R, _ = cv2.Rodrigues(rvec)
                camera_pos = -R.T @ tvec.flatten()
                camera_positions.append(camera_pos)
            
            camera_positions = np.array(camera_positions)
            fig.add_trace(go.Scatter3d(
                x=camera_positions[:, 0], y=camera_positions[:, 1], z=camera_positions[:, 2],
                mode='markers',
                marker=dict(size=8, color='blue', symbol='diamond'),
                name='Cameras'
            ))
            
            fig.update_layout(
                title='3D Camera Poses and Chessboard',
                scene=dict(
                    xaxis_title='X (m)',
                    yaxis_title='Y (m)',
                    zaxis_title='Z (m)'
                ),
                width=800,
                height=600
            )
            
            return fig


    class Calib:
        """Main calibration class"""
        
        def __init__(self, pattern_size: Tuple[int, int] = (9, 6), square_size: float = 0.025):
            self.board = Board(pattern_size, square_size)
            self.object_points = []
            self.image_points = []
            self.image_size = None
            self.camera_matrix = None
            self.dist_coeffs = None
            self.rvecs = None
            self.tvecs = None
            self.calibration_error = None
        
        def process_images(self, image_paths: List[str]) -> Tuple[int, int]:
            """Process calibration images to find chessboard corners"""
            self.object_points = []
            self.image_points = []
            successful_detections = 0
            
            for image_path in image_paths:
                image = Img.load_image(image_path)
                if image is None:
                    continue
                    
                if self.image_size is None:
                    self.image_size = (image.shape[1], image.shape[0])
                
                ret, corners = Board.find_corners(image, self.board.pattern_size)
                
                if ret:
                    self.object_points.append(self.board.objp)
                    self.image_points.append(corners)
                    successful_detections += 1
            
            return successful_detections, len(image_paths)
        
        def calibrate(self) -> bool:
            """Run camera calibration"""
            if len(self.object_points) < 10:
                return False
            
            ret, self.camera_matrix, self.dist_coeffs, self.rvecs, self.tvecs = Cam.calibrate_camera(
                self.object_points, self.image_points, self.image_size)
            
            if ret:
                self.calibration_error = Cam.compute_reprojection_error(
                    self.object_points, self.image_points, self.camera_matrix, 
                    self.dist_coeffs, self.rvecs, self.tvecs)
                return True
            
            return False
        
        def save_results(self, filename: str) -> None:
            """Save calibration results"""
            if self.camera_matrix is not None:
                IO.save_calibration_results(filename, self.camera_matrix, self.dist_coeffs, 
                                          self.rvecs, self.tvecs, self.image_size)
        
        def get_sample_images_with_axes(self, image_paths: List[str], num_samples: int = 5) -> List[np.ndarray]:
            """Get sample images with coordinate axes drawn"""
            if not self.rvecs or not self.tvecs:
                return []
            
            sample_images = []
            indices = np.linspace(0, len(image_paths) - 1, min(num_samples, len(image_paths)), dtype=int)
            
            for idx in indices:
                if idx < len(self.rvecs):
                    image = Img.load_image(image_paths[idx])
                    if image is not None:
                        img_with_axes = Overlay.draw_axes(
                            image, self.camera_matrix, self.dist_coeffs,
                            self.rvecs[idx], self.tvecs[idx], self.board.square_size * 3)
                        sample_images.append(img_with_axes)
            
            return sample_images

    print("✅ Helper classes loaded inline successfully!")
else:
    print("✅ Helper classes already available from external file!")

✅ Helper classes already available from external file!


## Gradio Interface Functions

These functions handle the Gradio UI interactions:

In [12]:
# Global calibration object
calibrator = None
current_image_paths = []

def process_uploaded_files(files, pattern_width, pattern_height, square_size):
    """Process uploaded calibration images"""
    global calibrator, current_image_paths
    
    if not files:
        return "No files uploaded!", None, None, None, None
    
    # Create images directory
    images_dir = "/content/images" if 'COLAB_GPU' in os.environ else "./images"
    IO.create_directory(images_dir)
    
    # Clear previous images
    for f in glob.glob(os.path.join(images_dir, "*")):
        if os.path.isfile(f):
            os.remove(f)
    
    # Save uploaded files
    current_image_paths = []
    for i, file in enumerate(files):
        if file is not None:
            file_path = os.path.join(images_dir, f"calibration_{i:03d}.jpg")
            shutil.copy(file.name, file_path)
            current_image_paths.append(file_path)
    
    # Initialize calibrator
    pattern_size = (int(pattern_width), int(pattern_height))
    calibrator = Calib(pattern_size, float(square_size))
    
    # Process images to detect corners
    successful, total = calibrator.process_images(current_image_paths)
    
    status = f"Processed {total} images, detected chessboard in {successful} images."
    
    if successful < 10:
        status += " Need at least 10 successful detections for calibration."
        return status, None, None, None, None
    
    # Show a sample image with detected corners
    sample_image = None
    if current_image_paths:
        img = Img.load_image(current_image_paths[0])
        ret, corners = Board.find_corners(img, pattern_size)
        if ret:
            sample_image = Overlay.draw_chessboard_corners(img, corners, pattern_size, ret)
            sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
    
    return status, sample_image, None, None, None


def run_calibration():
    """Run camera calibration"""
    global calibrator
    
    if calibrator is None:
        return "Please upload images first!", None, None, None, None
    
    if len(calibrator.object_points) < 10:
        return "Need at least 10 images with detected chessboard patterns!", None, None, None, None
    
    # Run calibration
    success = calibrator.calibrate()
    
    if not success:
        return "Calibration failed!", None, None, None, None
    
    # Save results
    calibration_file = "calibration.json"
    calibrator.save_results(calibration_file)
    
    # Generate results
    fx = calibrator.camera_matrix[0, 0]
    fy = calibrator.camera_matrix[1, 1]
    cx = calibrator.camera_matrix[0, 2]
    cy = calibrator.camera_matrix[1, 2]
    
    results_text = f"""Calibration Results:
    
Camera Matrix (K):
fx = {fx:.2f}, fy = {fy:.2f}
cx = {cx:.2f}, cy = {cy:.2f}

Distortion Coefficients:
{calibrator.dist_coeffs.flatten()}

Reprojection Error: {calibrator.calibration_error:.4f} pixels

Results saved to: {calibration_file}
    """
    
    # Create 3D plot
    fig = Render.plot_camera_poses_plotly(calibrator.rvecs, calibrator.tvecs, 
                                        calibrator.board.pattern_size, calibrator.board.square_size)
    
    # Get sample images with axes
    sample_images = calibrator.get_sample_images_with_axes(current_image_paths, 5)
    
    # Convert sample images to RGB for display
    rgb_samples = []
    for img in sample_images[:3]:  # Show first 3 samples
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        rgb_samples.append(rgb_img)
    
    # Pad with None if we have fewer than 3 images
    while len(rgb_samples) < 3:
        rgb_samples.append(None)
    
    # Create undistortion comparison
    undistort_comparison = None
    if current_image_paths:
        original = Img.load_image(current_image_paths[0])
        undistorted = Img.undistort_image(original, calibrator.camera_matrix, calibrator.dist_coeffs)
        
        # Resize for display
        original_resized = Img.resize_image(original, 400)
        undistorted_resized = Img.resize_image(undistorted, 400)
        
        # Combine side by side
        comparison = np.hstack([original_resized, undistorted_resized])
        undistort_comparison = cv2.cvtColor(comparison, cv2.COLOR_BGR2RGB)
    
    return results_text, fig, rgb_samples[0], rgb_samples[1], rgb_samples[2], undistort_comparison


def create_gradio_interface():
    """Create the Gradio interface"""
    
    with gr.Blocks(title="Camera Calibration", theme=gr.themes.Soft()) as interface:
        gr.Markdown("""
        # 📷 Camera Calibration with OpenCV
        
        Upload your chessboard calibration images and compute camera intrinsic parameters!
        
        **Instructions:**
        1. Print the chessboard pattern: [pattern.png](https://docs.opencv.org/4.x/pattern.png)
        2. Capture 15-20 images of the pattern from different angles
        3. Upload the images below and set the pattern parameters
        4. Click "Run Calibration" to compute camera parameters
        """)
        
        with gr.Row():
            with gr.Column(scale=2):
                gr.Markdown("### 📁 Upload Calibration Images")
                file_input = gr.Files(label="Upload Images (.jpg/.jpeg)", file_types=[".jpg", ".jpeg"])
                
                gr.Markdown("### ⚙️ Pattern Parameters")
                with gr.Row():
                    pattern_width = gr.Number(label="Pattern Width (inner corners)", value=9, precision=0)
                    pattern_height = gr.Number(label="Pattern Height (inner corners)", value=6, precision=0)
                square_size = gr.Number(label="Square Size (meters)", value=0.025, precision=4)
                
                process_btn = gr.Button("📋 Process Images", variant="secondary")
                calibrate_btn = gr.Button("🎯 Run Calibration", variant="primary")
            
            with gr.Column(scale=1):
                gr.Markdown("### 📊 Status")
                status_text = gr.Textbox(label="Status", lines=3, interactive=False)
                sample_detection = gr.Image(label="Sample Corner Detection")
        
        gr.Markdown("### 📈 Calibration Results")
        results_text = gr.Textbox(label="Results", lines=10, interactive=False)
        
        gr.Markdown("### 🎨 Visualizations")
        
        with gr.Tab("3D Camera Poses"):
            plot_3d = gr.Plot(label="Camera Poses and Chessboard")
        
        with gr.Tab("Sample Images with Axes"):
            with gr.Row():
                sample_img1 = gr.Image(label="Sample 1")
                sample_img2 = gr.Image(label="Sample 2")
                sample_img3 = gr.Image(label="Sample 3")
        
        with gr.Tab("Undistortion Comparison"):
            gr.Markdown("**Left: Original | Right: Undistorted**")
            undistort_img = gr.Image(label="Before/After Undistortion")
        
        # Event handlers
        process_btn.click(
            fn=process_uploaded_files,
            inputs=[file_input, pattern_width, pattern_height, square_size],
            outputs=[status_text, sample_detection, plot_3d, sample_img1, sample_img2]
        )
        
        calibrate_btn.click(
            fn=run_calibration,
            inputs=[],
            outputs=[results_text, plot_3d, sample_img1, sample_img2, sample_img3, undistort_img]
        )
    
    return interface

print("Gradio interface functions defined!")

Gradio interface functions defined!


## Launch the Gradio Interface

Run this cell to start the interactive calibration interface:

In [13]:
# Create and launch the interface
interface = create_gradio_interface()

# Launch with appropriate settings for different environments
if 'COLAB_GPU' in os.environ:
    # Running in Google Colab
    interface.launch(share=True, debug=True)
else:
    # Running locally
    interface.launch(share=False, debug=True, server_port=7860)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


## Manual Testing (Optional)

You can also test the calibration functions manually:

In [ ]:
# Manual testing example
# Uncomment and modify paths to test with your own images

# test_calibrator = Calib(pattern_size=(9, 6), square_size=0.025)
# test_images = ["path/to/image1.jpg", "path/to/image2.jpg"]  # Add your image paths
# 
# successful, total = test_calibrator.process_images(test_images)
# print(f"Processed {total} images, detected chessboard in {successful} images")
# 
# if successful >= 10:
#     success = test_calibrator.calibrate()
#     if success:
#         print("Calibration successful!")
#         print(f"Camera matrix:\n{test_calibrator.camera_matrix}")
#         print(f"Distortion coefficients: {test_calibrator.dist_coeffs.flatten()}")
#         print(f"Reprojection error: {test_calibrator.calibration_error:.4f} pixels")
#         
#         test_calibrator.save_results("manual_calibration.json")
#         print("Results saved to manual_calibration.json")

## Summary

This notebook provides a complete camera calibration system with:

1. **Modular Design**: Organized into helper classes (IO, Board, Img, Overlay, Cam, Render, Calib)
2. **Interactive UI**: Gradio interface for easy use
3. **Comprehensive Visualization**: 
   - 3D camera pose plots
   - Sample images with coordinate axes
   - Undistortion comparison
4. **File I/O**: Saves calibration results to JSON format
5. **Error Handling**: Robust processing with status updates

**Usage Steps:**
1. Print the OpenCV chessboard pattern
2. Capture 15-20 calibration images
3. Upload images through the interface
4. Set pattern parameters (default: 9x6 inner corners, 25mm squares)
5. Process images to detect chessboard corners
6. Run calibration to compute camera parameters
7. View results and visualizations

The calibration results are saved to `calibration.json` and include the camera matrix (K), distortion coefficients, and reprojection error.